### Wczytanie pakietów oraz danych

In [3]:
import pandas as pd
import numpy as np

In [4]:
# https://drive.google.com/drive/folders/1A7dwlkOiW2BYmZWQmM_9emP6ia7SPv9U?usp=sharing
Posts = pd.read_csv("Dane/Posts.csv.gz", compression = 'gzip')
Users = pd.read_csv("Dane/Users.csv.gz", compression = 'gzip')
PostLinks = pd.read_csv("Dane/PostLinks.csv.gz", compression = 'gzip')
Comments = pd.read_csv("Dane/Comments.csv.gz", compression = 'gzip')

### Tworze polaczenie do bazy danych

In [5]:
import os, os.path
import sqlite3
import tempfile

# sciezka dostepu do bazy danych:
baza = os.path.join(tempfile.mkdtemp(), 'przyklad.db')
if os.path.isfile(baza): # jesli baza juz istneje
    os.remove(baza) # usuniemy ja
    
conn = sqlite3.connect(baza) # połaczenie do bazy danych

In [6]:
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
print("")

### Import rozwiązań z pliku

In [7]:
from functions import *

### Zadanie 1

In [8]:
df_sql_1 = pd.read_sql_query(
    """
    SELECT Location, COUNT(*) AS Count
    FROM (
    SELECT Posts.OwnerUserId, Users.Id, Users.Location
    FROM Users
    JOIN Posts ON Users.Id = Posts.OwnerUserId
    )
    WHERE Location NOT IN ('')
    GROUP BY Location
    ORDER BY Count DESC
    LIMIT 10
    """, conn)

In [9]:
# Import rozwiązan z pliku py
df_pandas_1 = solution_1(Posts, Users)

### Porównanie wyników

In [10]:
df_sql_1.equals(df_pandas_1)

True

## Zadanie 2

In [11]:
df_sql_2 = pd.read_sql_query(
    """
    SELECT Posts.Title, RelatedTab.NumLinks
    FROM
    (
    SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
    FROM PostLinks
    GROUP BY RelatedPostId
    ) AS RelatedTab
    JOIN Posts ON RelatedTab.PostId=Posts.Id
    WHERE Posts.PostTypeId=1
    ORDER BY NumLinks DESC

    """, conn)

In [12]:
# Import rozwiązan z pliku py
df_pandas_2 = solution_2(Posts, PostLinks)

### Porównanie wyników

In [13]:
df_sql_2.equals(df_pandas_2)

True

### Zadanie 3

In [14]:
df_sql_3 = pd.read_sql_query(
    """
    SELECT Title, CommentCount, ViewCount, CommentsTotalScore,
    DisplayName, Reputation, Location
    FROM (
    SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
    CmtTotScr.CommentsTotalScore
    FROM (
    SELECT PostId, SUM(Score) AS CommentsTotalScore
    FROM Comments
    GROUP BY PostId
    ) AS CmtTotScr
    JOIN Posts ON Posts.Id = CmtTotScr.PostId
    WHERE Posts.PostTypeId=1
    ) AS PostsBestComments
    JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
    ORDER BY CommentsTotalScore DESC
    LIMIT 10

    """, conn)

In [15]:
# Import rozwiązan z pliku py
df_pandas_3 = solution_3(Comments, Posts, Users)

### Porównanie wyników

In [16]:
df_sql_3.equals(df_pandas_3)

True

### Zadanie 4

In [17]:
df_sql_4 = pd.read_sql_query(
    """
    SELECT DisplayName, QuestionsNumber, AnswersNumber, Location,
    Reputation, UpVotes, DownVotes
    FROM (
    SELECT *
    FROM (
    SELECT COUNT(*) as AnswersNumber, OwnerUserId
    FROM Posts
    WHERE PostTypeId = 2
    GROUP BY OwnerUserId
    ) AS Answers
    JOIN
    (
    SELECT COUNT(*) as QuestionsNumber, OwnerUserId
    FROM Posts
    WHERE PostTypeId = 1
    GROUP BY OwnerUserId
    ) AS Questions
    ON Answers.OwnerUserId = Questions.OwnerUserId
    WHERE AnswersNumber > QuestionsNumber
    ORDER BY AnswersNumber DESC
    LIMIT 5
    ) AS PostsCounts
    JOIN Users ON PostsCounts.OwnerUserId = Users.Id
    
    """, conn)

In [18]:
# Import rozwiązan z pliku py
df_pandas_4 = solution_4(Posts, Users)

### Porównanie wyników

In [19]:
df_sql_4.equals(df_pandas_4)

True

### Zadanie 5

In [20]:
df_sql_5 = pd.read_sql_query(
    """
    SELECT
    Users.AccountId,
    Users.DisplayName,
    Users.Location,
    AVG(PostAuth.AnswersCount) as AverageAnswersCount
    FROM
    (
    SELECT
    AnsCount.AnswersCount,
    Posts.Id,
    Posts.OwnerUserId
    FROM (
    SELECT Posts.ParentId, COUNT(*) AS AnswersCount
    FROM Posts
    WHERE Posts.PostTypeId = 2
    GROUP BY Posts.ParentId
    ) AS AnsCount
    JOIN Posts ON Posts.Id = AnsCount.ParentId
    ) AS PostAuth
    JOIN Users ON Users.AccountId=PostAuth.OwnerUserId
    GROUP BY OwnerUserId
    ORDER BY AverageAnswersCount DESC
    LIMIT 10
    
    """, conn)

In [21]:
# Import rozwiązan z pliku py
df_pandas_5 = solution_5(Posts, Users)

### Porównanie wyników

In [22]:
df_sql_5.equals(df_pandas_5)

True